<table>
    <tr>
        <td><img src="https://s3.amazonaws.com/media-p.slid.es/uploads/1485763/images/9060062/Header.png" width="300"/></td>
        <td>&nbsp;</td>
        <td>
            <h1 style="font-size:200%;color:blue;text-align:center">    <FONT COLOR="blue">  Generación 3D  </FONT>         </h1></td>         
        <td>
            <tp><p style="font-size:99%;text-align:center">Visión por computadora </p></tp>
            <tp><p style="font-size:115%;text-align:center">Pregrado MACC 2024-2</p></tp>
            <tp><p style="font-size:115%;text-align:center">Nicolás Dussan & Nicolas Eduardo Gallego</p></tp>
        </td>
    </tr>
</table>

**Importación de librearias**

In [ ]:
!pip install open3d
import cv2 as cv
import glob
import os
import numpy as np
import open3d as o3d

**Calibración de la cámara**

In [ ]:
#definir una función para la calibración de la cámara a partir de las imagenes de chess
def calibración(iter, umbral, dir):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, iter, umbral)

    objp = np.zeros((7 * 7, 3), np.float32)
    objp[:, :2] = np.mgrid[0:7, 0:7].T.reshape(-1, 2)

    objpoints = []
    imgpoints = []

    imagenes = dir

    for imagen in imagenes:
        img = cv.imread(imagen)
        if img is None:
            print(f" No se pudo cargar la imagen {imagen}")
            continue

        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        ret, corners = cv.findChessboardCorners(gray, (7, 7), None)

        if ret:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
        else:
            print(f"No se detectaron esquinas en {imagen}")
    cv.destroyAllWindows()

    if len(objpoints) > 10:
        ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        return ret, mtx, dist, rvecs, tvecs
#cambiar la ruta o dirrección donde se encuentran las imagenes de chess1
chess = glob.glob(r"C:\Users\nicol\OneDrive\Escritorio\chess1\*.jpg")
ret, mtxk, distr, rvecs, tvecs = calibración(30, 0.0001, chess)

TypeError: cannot unpack non-iterable NoneType object

**Corrección de distorsión**

In [ ]:
def corregir_distorsion(ruta_entrada, ruta_salida, mtx, dist):
    if not os.path.exists(ruta_salida):
        os.makedirs(ruta_salida)
    for archivo in os.listdir(ruta_entrada):
        if archivo.endswith((".jpg", ".png", ".jpeg")):
            ruta_original = os.path.join(ruta_entrada, archivo)
            img = cv.imread(ruta_original)
            img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
            h, w = img.shape[:2]
            new_mtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))
            img_corregida = cv.undistort(img, mtx, dist, None, new_mtx)
            ruta_guardado = os.path.join(ruta_salida, archivo)
            cv.imwrite(ruta_guardado, img_corregida)
            print(f"✅ Imagen corregida: {archivo}")

#cambiar ruta o dirección a donde se enceuntran las imagenes del objeto
camaro = "C:/Users/nicol/OneDrive/Escritorio/camaro1"
salida = os.path.join(camaro, "mejoradas")
corregir_distorsion(camaro, salida, mtxk, distr)

**Empajeramientos por caracteristicas de la imagen**

In [ ]:
salida1 = os.path.join(camaro, "emparejamientos")

# Se crea el directorio de salida en caso de que no exista
if not os.path.exists(salida1):
    os.makedirs(salida1)


sift = cv.SIFT_create()

# Usamos el matcher de fuerza bruta con la nomra L2
bf = cv.BFMatcher(cv.NORM_L2, crossCheck=False)

# Función de emparejamiento de descriptores
def emparejar_descriptores(desc1, desc2):
    matches = bf.knnMatch(desc1, desc2, k=2)
    buenos_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    return buenos_matches
# Para esta funcion usamos la prueba lowe

# Obtener las imágenes de la carpeta
imagenes = [os.path.join(salida, f) for f in os.listdir(salida) if f.endswith(('.jpg', '.png', '.jpeg'))]
imagenes.sort()  # Asegurar orden correcto

# Detectar características en todas las imágenes
keypoints_descriptors = []

for img_path in imagenes:
    img = cv.imread(img_path)
    keypoints, descriptors = sift.detectAndCompute(img, None)
    keypoints_descriptors.append((img_path, keypoints, descriptors))
    print(f"🔍 Procesada: {img_path} - Keypoints: {len(keypoints)}")


rango = 2

for i in range(len(keypoints_descriptors)):
    img1_path, kp1, desc1 = keypoints_descriptors[i]
    for j in range(i + 1, min(i + rango + 1, len(keypoints_descriptors))):
        img2_path, kp2, desc2 = keypoints_descriptors[j]
        buenos_matches = emparejar_descriptores(desc1, desc2)
        print(f"🔗 Emparejadas: {img1_path} ↔ {img2_path} | Matches: {len(buenos_matches)}")
        img1 = cv.imread(img1_path)
        img2 = cv.imread(img2_path)
        img_match = cv.drawMatches(img1, kp1, img2, kp2, buenos_matches, None, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
        nombre_salida = f"match_{i}_con_{j}.png"
        cv.imwrite(os.path.join(salida1, nombre_salida), img_match)
        cv.namedWindow("Emparejamientos", cv.WINDOW_NORMAL)
        cv.resizeWindow("Emparejamientos", 2560, 600)
        cv.imshow("Emparejamiento", img_match)
        cv.waitKey(500)

cv.destroyAllWindows()
print("✅ ¡Emparejamiento ampliado completado!")

**Matrices Fundamental, Escencial, Rotación y Traslación**

In [ ]:
# Calcular matrices (Fundamental, Esencial, Rotación y Traslación)
def calcular_matrices(kp1, desc1, kp2, desc2, K):
    buenos_matches = emparejar_descriptores(desc1, desc2)

    if len(buenos_matches) < 8:
        print("⚠️ Advertencia: No hay suficientes puntos para calcular las matrices.")
        return None

    # Extraer puntos correspondientes
    pts1 = np.float32([kp1[m.queryIdx].pt for m in buenos_matches])
    pts2 = np.float32([kp2[m.trainIdx].pt for m in buenos_matches])

    # Calcular la matriz fundamental (F) usando RANSAC
    F, _ = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC, 1.0, 0.999)
    print("Matriz Fundamental (F):\n", F)

    # Calcular la matriz esencial (E) con los parámetros de calibración
    E, _ = cv.findEssentialMat(pts1, pts2, K, method=cv.RANSAC, prob=0.999, threshold=1.0)
    print("Matriz Esencial (E):\n", E)

    # Descomponer la matriz esencial en R (rotación) y t (traslación)
    _, R, t, _ = cv.recoverPose(E, pts1, pts2, K)
    t = t / np.linalg.norm(t)  # Normalizar la traslación

    print("Matriz de Rotación (R):\n", R)
    print("Vector de Traslación (t, normalizado):\n", t)

    return F, E, R, t

# Calcular las matrices para imágenes consecutivas
for i in range(len(keypoints_descriptors) - 1):
    img1_path, kp1, desc1 = keypoints_descriptors[i]
    img2_path, kp2, desc2 = keypoints_descriptors[i + 1]

    print(f"\n🔢 Calculando matrices para: {img1_path} ↔ {img2_path}")
    calcular_matrices(kp1, desc1, kp2, desc2, mtxk)

**Visualización del modelo de colmap con open3d**

In [ ]:
# Ruta del archivo PLY exportado
ply_path = "C:/Users/nicol/OneDrive/Escritorio/camaro1/camaro.ply"


nube = o3d.io.read_point_cloud(ply_path)
print(nube)
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.add_geometry(nube)

# Ajustar el tamaño de los puntos
opt = vis.get_render_option()
opt.point_size = 1.0
vis.run()
vis.destroy_window()